# Modeling Your Data - Lab

## Introduction 

In this lab you'll perform a full linear regression on the data. You'll implement the process demonstrated in the previous lesson, taking a stepwise approach to analyze and improve the model along the way.

## Objectives
You will be able to:

* Remove predictors with p-values too high and refit the model
* Examine and interpret the model results
* Split data into training and testing sets
* Fit a regression model to the data set using statsmodel library


## Build an Initial Regression Model

To start, perform a train-test split and create an initial regression model to model the `list_price` using all of your available features.

> **Note:** In order to write the model you'll have to do some tedious manipulation of your column names. Statsmodels will not allow you to have spaces, apostrophe or arithmetic symbols (+) in your column names. Preview them and refine them as you go.  
**If you receive an error such as "PatsyError: error tokenizing input (maybe an unclosed string?)" then you need to further preprocess your column names.**

In [2]:
#Your code here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv('Lego_dataset_cleaned.csv')

In [5]:
df.head()

,piece_count,list_price,num_reviews,play_star_rating,star_rating,val_star_rating,ages_10+,ages_10-14,ages_10-16,ages_10-21,...,country_NZ,country_PL,country_PT,country_US,review_difficulty_Average,review_difficulty_Challenging,review_difficulty_Easy,review_difficulty_Very Challenging,review_difficulty_Very Easy,review_difficulty_unknown
0,-0.273020,29.99,-0.398512,-0.655279,-0.045687,-0.365010,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
1,-0.404154,19.99,-0.398512,-0.655279,0.990651,-0.365010,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
2,-0.517242,12.99,-0.147162,-0.132473,-0.460222,-0.204063,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
3,0.635296,99.99,0.187972,-1.352353,0.161581,0.117830,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
4,0.288812,79.99,-0.063378,-2.049427,0.161581,-0.204063,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0


In [29]:
subs = [(' ', '_'),('.',''),("'",""),('™', ''), ('®',''),
        ('+','plus'), ('½','half'), ('-','_')
       ]
def col_formatting(col):
    for old, new in subs:
        col = col.replace(old,new)
    return col

In [30]:
df.columns = [col_formatting(col) for col in df.columns]


In [31]:
list_price= 'list_price'

In [32]:
cols = list(df.columns)

In [33]:
cols.remove('list_price')

In [34]:
cols

['piece_count',
 'num_reviews',
 'play_star_rating',
 'star_rating',
 'val_star_rating',
 'ages_10plus',
 'ages_10_14',
 'ages_10_16',
 'ages_10_21',
 'ages_11_16',
 'ages_12plus',
 'ages_12_16',
 'ages_14plus',
 'ages_16plus',
 'ages_1half_3',
 'ages_1half_5',
 'ages_2_5',
 'ages_4plus',
 'ages_4_7',
 'ages_4_99',
 'ages_5plus',
 'ages_5_12',
 'ages_5_8',
 'ages_6plus',
 'ages_6_12',
 'ages_6_14',
 'ages_7plus',
 'ages_7_12',
 'ages_7_14',
 'ages_8plus',
 'ages_8_12',
 'ages_8_14',
 'ages_9plus',
 'ages_9_12',
 'ages_9_14',
 'ages_9_16',
 'theme_name_Angry_Birds',
 'theme_name_Architecture',
 'theme_name_BOOST',
 'theme_name_Blues_Helicopter_Pursuit',
 'theme_name_BrickHeadz',
 'theme_name_Carnotaurus_Gyrosphere_Escape',
 'theme_name_City',
 'theme_name_Classic',
 'theme_name_Creator_3_in_1',
 'theme_name_Creator_Expert',
 'theme_name_DC_Comics_Super_Heroes',
 'theme_name_DC_Super_Hero_Girls',
 'theme_name_DIMENSIONS',
 'theme_name_DUPLO',
 'theme_name_Dilophosaurus_Outpost_Attack',
 

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10870 entries, 0 to 10869
Columns: 103 entries, piece_count to review_difficulty_unknown
dtypes: float64(6), int64(97)
memory usage: 8.5 MB


In [36]:
train, test = train_test_split(df)

In [37]:
print(len(train), len(test))
train.head()

8152 2718


,piece_count,list_price,num_reviews,play_star_rating,star_rating,val_star_rating,ages_10plus,ages_10_14,ages_10_16,ages_10_21,...,country_NZ,country_PL,country_PT,country_US,review_difficulty_Average,review_difficulty_Challenging,review_difficulty_Easy,review_difficulty_Very_Challenging,review_difficulty_Very_Easy,review_difficulty_unknown
399,-0.485962,19.9900,-0.398512,1.087407,0.990651,0.439724,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
5359,-0.056468,79.2878,-0.119234,0.041796,-0.460222,-1.008797,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
10169,-0.395732,40.5971,-0.398512,1.087407,0.990651,-0.365010,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
3059,-0.141885,40.6980,-0.175090,-2.049427,-1.496559,-0.525957,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4576,-0.424606,30.0000,-0.426440,1.087407,0.990651,1.244458,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [38]:
predictors = '+'.join(cols)
formula = list_price + "~" + predictors
model = ols(formula=formula, data=train).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             list_price   R-squared:                       0.864
Model:                            OLS   Adj. R-squared:                  0.863
Method:                 Least Squares   F-statistic:                     552.7
Date:                Mon, 17 Jun 2019   Prob (F-statistic):               0.00
Time:                        21:32:40   Log-Likelihood:                -40424.
No. Observations:                8152   AIC:                         8.104e+04
Df Residuals:                    8058   BIC:                         8.169e+04
Df Model:                          93                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                           63.5366      1.712     37.107      0.000      60.180      66.893
piece_count                                         75.9405      0.882     86.099      0.000      74.211      77.669
num_reviews                                          7.2120      0.689     10.468      0.000       5.861       8.562
play_star_rating                                     5.1635      0.616      8.383      0.000       3.956       6.371
star_rating                                         -1.0929      0.699     -1.564      0.118      -2.463       0.277
val_star_rating                                     -8.3706      0.622    -13.461      0.000      -9.590      -7.152
ages_10plus                                        123.9450      6.429     19.279      0.000     111.343     136.547
ages_10_14                                         -25.9868      8.100     -3.208      0.001     -41.864     -10.110
ages_10_16                                         -10.6458      3.894     -2.734      0.006     -18.278      -3.013
ages_10_21                                          57.6459      7.229      7.974      0.000      43.475      71.816
ages_11_16                                         -22.7904      5.488     -4.152      0.000     -33.549     -12.032
ages_12plus                                         68.7926      5.228     13.159      0.000      58.544      79.041
ages_12_16                                         -58.7761      6.946     -8.462      0.000     -72.391     -45.161
ages_14plus                                         23.5802      3.989      5.911      0.000      15.760      31.400
ages_16plus                                         28.0377      5.834      4.806      0.000      16.602      39.474
ages_1half_3                                        -8.6338      2.353     -3.669      0.000     -13.247      -4.021
ages_1half_5                                        -3.9617      2.974     -1.332      0.183      -9.791       1.867
ages_2_5                                            15.1366      1.657      9.135      0.000      11.889      18.385
ages_4plus                                         -20.0114      6.610     -3.027      0.002     -32.969      -7.054
ages_4_7                                            -4.8504      3.066     -1.582      0.114     -10.861       1.160
ages_4_99                                           -2.3623      3.714     -0.636      0.525      -9.643       4.918
ages_5plus                                          -9.4278      2.993     -3.150      0.002     -15.294      -3.561
ages_5_12                                          -36.1416      2.991    -12.084      0.000     -42.004     -30.279
ages_5_8                                 

## Remove the Uninfluential Features

Based on the initial model, remove those features which do not appear to be statistically relevant and rerun the model.

In [39]:
#Your code here
summary = model.summary()
p_table = summary.tables[1]
p_table = pd.DataFrame(p_table.data)
p_table.columns = p_table.iloc[0]
p_table = p_table.drop(0)
p_table = p_table.set_index(p_table.columns[0])
p_table['P>|t|'] = p_table['P>|t|'].astype(float)
x_cols = list(p_table[p_table['P>|t|']<0.05].index)
x_cols.remove('Intercept')
print(len(p_table), len(x_cols))
print(x_cols[:5])
p_table.head()

103 74
['piece_count', 'num_reviews', 'play_star_rating', 'val_star_rating', 'ages_10plus']


,coef,std err,t,P>|t|,[0.025,0.975]
,,,,,,
Intercept,63.5366,1.712,37.107,0.000,60.180,66.893
piece_count,75.9405,0.882,86.099,0.000,74.211,77.669
num_reviews,7.2120,0.689,10.468,0.000,5.861,8.562
play_star_rating,5.1635,0.616,8.383,0.000,3.956,6.371
star_rating,-1.0929,0.699,-1.564,0.118,-2.463,0.277


> **Comment:** You should see that the model performance is identical. Additionally, observe that there are further features which have been identified as unimpactful. Continue to refine the model accordingly.

In [ ]:
#Your code here

## Investigate Multicollinearity

There are still a lot of features in the current model! Chances are there are some strong multicollinearity issues. Begin to investigate the extend of this problem.

In [ ]:
#Your code here

## Perform Another Round of Feature Selection

Once again, subset your features based on your findings above. Then rerun the model once again.

In [ ]:
#Your code here

## Check the Normality Assumption

Check whether the normality assumption holds for your model.

In [ ]:
# Your code here

## Check Homoscedasticity Assumption

Check whether the model's errors are indeed homoscedastic or if they violate this principle and display heteroscedasticity.

In [ ]:
#Your code here

> **Comment:** This displays a fairly pronounced 'funnel' shape: errors appear to increase as the list_price increases. This doesn't bode well for our model. Subsetting the data to remove outliers and confiding the model to this restricted domain may be necessary. A log transformation or something equivalent may also be appropriate.

## Make Additional Refinements

From here, make additional refinements to your model based on the above analysis. As you progress, continue to go back and check the assumptions for the updated model. Be sure to attempt at least 2 additional model refinements.

> **Comment:** Based on the above plots, it seems as though outliers are having a substantial impact on the model. As such, removing outliers may be appropriate. Investigating the impact of a log transformation is also worthwhile.

In [ ]:
#Your code here

## Summary

Well done! As you can see, regression can be a challenging task that requires you to make decisions along the way, try alternative approaches and make ongoing refinements. These choices depend on the context and specific use cases. 